In [34]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import yaml
from types import SimpleNamespace

import sys
from pathlib import Path

# 1) figure out your project root (one level up from this notebook)
project_root = Path().resolve().parent

# 2) add it to the front of sys.path
sys.path.insert(0, str(project_root))

from src.models.moclip import build_moclip
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Donwload MOCLIP
# Load YAML config file
with open('../src/models/config.yaml', 'r') as file:
    cfg_dict = yaml.safe_load(file)

# Convert to SimpleNamespace for easy attribute access
cfg = SimpleNamespace(**cfg_dict)

moclip = build_moclip(cfg)
moclip = moclip.to(device)

In [ ]:
# Get Geometry and Spectra encoders
geometry_encoder = moclip.geometry_encoder
spectra_encoder = moclip.spectra_encoder

In [ ]:
# Get test dataset
root = "../data/MOCLIP_dataset_10k.npz"
Dataset = np.load(root, allow_pickle=True)
wvl = Dataset['wvl'] # Shape (29,) wavelengths in nm

geom = torch.from_numpy(Dataset['geom']) # Shape (Batch, 128, 128)
params = torch.from_numpy(Dataset['params']) # Shape (Batch, 2) (period, thickness) in um
spectra = torch.from_numpy(np.concatenate([Dataset['spectra_x'], Dataset['spectra_y']], axis=1)) # Shape (Batch, 58)

In [ ]:
# Get embeddings
geom_emb = geometry_encoder(geom.to(device)) # Shape (Batch, 64)
spectra_emb = spectra_encoder(spectra.to(device)) # Shape (Batch, 64)

# **MUST NORMALIZE EMBEDDINGS**
geom_emb = F.normalize(geom_emb, p=2, dim=1)
spectra_emb = F.normalize(spectra_emb, p=2, dim=1)

array([450., 460., 470., 480., 490., 500., 510., 520., 530., 540., 550.,
       560., 570., 580., 590., 600., 610., 620., 630., 640., 650., 660.,
       670., 680., 690., 700., 710., 720., 730.])